In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.1.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Vine_Review_Analysis').config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Review data into DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

### Populate Vine DataFrame

In [4]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
|R2XYDBMHUVJCSX|          2|            0|          1|   N|                Y|
|R1L94ESVC657A9|          5|            0|          0|   N|                Y|
|R3DI4B8LDWFQ3K|          1|            3|          4|   N|                Y|
|R1G7VV4HCXUQQL|          5|            0|          0|   N|                Y|
|R3TTNYN1TAMUSU|          5|            0|          0|   N|     

In [5]:
# Create DataFrame with total_votes >= 20
from pyspark.sql.functions import col

vine_gte_20_df = vine_df.filter(col('total_votes') >= 20)
vine_gte_20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|
|R1M8EEGH2TVS0Q|          1|            6|         29|   N|                Y|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|
|R3JF2SBRHZITRZ|          5|           24|         26|   N|                N|
| RLJN73T8WUYIR|          5|          124|        128|   N|     

In [9]:
# helpful_pct >= 50%
helpful_pct_df = vine_gte_20_df.withColumn('helpful_pct', col('helpful_votes')/col('total_votes'))
vine_helpful_pct_df = helpful_pct_df.filter(col('helpful_pct') >= 0.50)
vine_helpful_pct_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [10]:
# Paid reviews (vine == 'Y')
paid_reviews_df = vine_helpful_pct_df.filter(col('vine') == 'Y')
paid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RG70K1HMY4LXX|          4|           37|         44|   Y|                N|0.8409090909090909|
|R1DHGTNXDXJ0GB|          5|           78|         93|   Y|                N|0.8387096774193549|
|R3B5S3OSA8N6AO|          5|           38|         43|   Y|                N|0.8837209302325582|
|R29X0YBS0SBIUP|          5|           56|         74|   Y|                N|0.7567567567567568|
|R1ZEDMCU6NYC8Q|          3|           26|         27|   Y|                N|0.9629629629629629|
| RVBJM1LUI9X9K|          5|           27|         28|   Y|                N|0.9642857142857143|
| R1KASHD9QSDMV|          4|           18|         21|   Y|                N|0.8571428571428571|
|R1M3LATUI38MK0|          5|  

In [11]:
# Unpaid reviews (vine == 'N')
unpaid_reviews_df = vine_helpful_pct_df.filter(col('vine') == 'N')
unpaid_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|       helpful_pct|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2984F9VD9WDJB|          1|           37|         46|   N|                Y|0.8043478260869565|
|R218TGWCIAZYNY|          5|          191|        198|   N|                Y|0.9646464646464646|
| RGC2TLSBHLPZA|          5|           49|         50|   N|                Y|              0.98|
|R2YYVOW2RYGRM8|          5|           27|         28|   N|                Y|0.9642857142857143|
|R2X4WF89XICSB2|          1|           21|         21|   N|                Y|               1.0|
|R1M1KJ4SQE4INF|          5|           20|         21|   N|                N|0.9523809523809523|
| RK8MUEWEUGV6V|          4|          105|        113|   N|                N|0.9292035398230089|
|R3JF2SBRHZITRZ|          5|  

In [21]:
# Total number of reviews
total_number_reviews = vine_gte_20_df.count()
total_number_reviews

133416

In [27]:
# Total number of 5-star reviews
five_star_numb = vine_gte_20_df.filter(col('star_rating') == 5).count()
five_star_numb

76294

In [26]:
# Number of paid 5-star reviews
paid_5star_numb = paid_reviews_df.filter(col('star_rating') == 5).count()
paid_5star_numb

220

In [30]:
# Number of unpaid 5-star reviews
unpaid_5star_numb = unpaid_reviews_df.filter(col('star_rating') == 5).count()
unpaid_5star_numb

74470

In [29]:
# Percentage of paid 5-star reviews
paid_5star_pct = paid_5star_numb/ five_star_numb
paid_5star_pct

0.0028835819330484706

In [31]:
# Percentage of unpaid 5-star reviews
unpaid_5star_pct = unpaid_5star_numb / five_star_numb
unpaid_5star_pct

0.9760924843369072